In [1]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from numpy.linalg import norm
import numpy as np
import pickle

from tqdm import tqdm

In [2]:
model=ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3))
model.trainable=False
model=tensorflow.keras.Sequential([model,GlobalMaxPooling2D()])
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x000002346FBBE4C0>>

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [4]:
def extract_features(img_path,model):
    img=image.load_img(img_path,target_size=(224,224))
    img_arr=image.img_to_array(img)
    expand_img_arr= np.expand_dims(img_arr,axis=0)
    preprocessed_img=preprocess_input(expand_img_arr)
    result=model.predict(preprocessed_img).flatten()
    normalized_result=result/norm(result)
    
    return normalized_result

In [5]:
import os

In [6]:
filenames=[]
for file in os.listdir("C:\\Users\\hp\\Downloads\\images"):
   filenames.append(os.path.join("C:\\Users\\hp\\Downloads\\images",file))
print(len(filenames))
print(filenames[0:5])

44441
['C:\\Users\\hp\\Downloads\\images\\10000.jpg', 'C:\\Users\\hp\\Downloads\\images\\10001.jpg', 'C:\\Users\\hp\\Downloads\\images\\10002.jpg', 'C:\\Users\\hp\\Downloads\\images\\10003.jpg', 'C:\\Users\\hp\\Downloads\\images\\10004.jpg']


In [7]:
feature_list=[]
for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))
print(np.array(feature_list))


100%|██████████████████████████████████████████████████████████████████████████| 44441/44441 [2:38:51<00:00,  4.66it/s]


[[0.         0.01761616 0.00171594 ... 0.01247232 0.02726384 0.06899207]
 [0.         0.03648943 0.         ... 0.00997928 0.02375534 0.04649908]
 [0.         0.03507707 0.00732284 ... 0.00283519 0.         0.05435891]
 ...
 [0.         0.05594851 0.00553631 ... 0.00670328 0.01752765 0.00746652]
 [0.         0.02520914 0.00274131 ... 0.00833132 0.01841293 0.0042276 ]
 [0.         0.03215053 0.00704543 ... 0.00988327 0.02218141 0.00713839]]


In [8]:
print(np.array(feature_list))

[[0.         0.01761616 0.00171594 ... 0.01247232 0.02726384 0.06899207]
 [0.         0.03648943 0.         ... 0.00997928 0.02375534 0.04649908]
 [0.         0.03507707 0.00732284 ... 0.00283519 0.         0.05435891]
 ...
 [0.         0.05594851 0.00553631 ... 0.00670328 0.01752765 0.00746652]
 [0.         0.02520914 0.00274131 ... 0.00833132 0.01841293 0.0042276 ]
 [0.         0.03215053 0.00704543 ... 0.00988327 0.02218141 0.00713839]]


In [9]:
print(feature_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
pickle.dump(feature_list,open('embedding.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))